In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import train_test_split
import re
import statsmodels.api as sm
from sklearn import metrics
%matplotlib inline

## Importación Dataset

In [ ]:
df = pd.read_csv('properatti.csv', index_col = 0) ### Dataset Completo
## DataFrame Partido en distintas partes relevantes
df_posicion_geografica = df.loc[:,['place_name','state_name','country_name','place_with_parent_names' \
                                   ,'geonames_id','lat','lon','lat-lon', \
                                   'describe','title']] ### Parte con Posición Geográfica
df_localidad=df.loc[:,['place_name','place_with_parent_names','describe','title']] ## Parte con Localidades
df_ambientes = df.loc[:,['rooms','property_type','description','title']]  ## Parte con Ambientes
df_pisos = df.loc[:,['floor','property_type','description','title']]  ## Parte con Pisos
df_precio = df.loc[:,['price','currency','price_aprox_local_currency','price_aprox_usd',\
                      'surface_total_in_m2','surface_covered_in_m2','price_usd_per_m2',\
                      'price_per_m2','description','title']]  ## Parte con Precios y Superficies

## Funciones

In [ ]:
def crear_serie_precio(x):
    if not np.isnan(np.array(x.precio_en_titulo)):
        return x.precio_en_titulo
    elif not np.isnan(np.array(x.price_aprox_usd)):
        return x.price_aprox_usd
    else:
        return np.NaN
def crear_serie_ambientes(x):
    if not np.isnan(np.array(x.rooms_description)):
        return x.rooms_description
    if not np.isnan(np.array(x['rooms_title'])):
        return x['rooms_title']
    if not np.isnan(np.array(x.rooms)):
        return x.rooms
    return np.NaN
def detectar_ambientes(x):
    y = busqueda_monoambientes(x)
    if np.isnan(np.array(y)):
        y=busqueda_ambientes(x)
    if np.isnan(np.array(y)):
        y=busqueda_por_texto(x)
    return y
def update(df,column,s):
    '''
    Reemplaza los valores de la columna del DataFrame con
    los valores no nulos de la serie
    '''
    df[column]=s.fillna(df[column])
    return df

def busqueda_ambientes(x):
    '''
    Busca la palabra amb (de ambientes) y devuelve el número que tiene por delante
    '''
    try:
        pattern = '(\d+)\s*amb'
        regex = re.compile(pattern, flags = re.IGNORECASE | re.UNICODE)
        m = regex.search(x)
        if m:
            valor =float(m.group(1))
            if valor < 8:
                return valor
            else:
                return 1.0
        else:
            return np.NaN
    except:
        return np.NaN

def busqueda_monoambientes(x):
    '''
    Busca en la palabra monoambiente y devuelve 1
    '''
    if type(x) is str:
        if x.lower().find('monoambiente') >-1:
            return 1.0
        else:
            return np.NaN
    else:
        return np.NaN
def busqueda_moneda(x):
    try:
        pattern = 'U\$D\s*([\d\.]+)'
        regex = re.compile(pattern, flags = re.IGNORECASE | re.UNICODE)
        m = regex.search(x)
        if m:
            return float(m.group(1).replace('.',''))
        else:
            np.NaN
    except:
        return np.NaN
    
def parsear_numeros(x):
    '''
    Reemplaza numeros escritos en letras por sus equivalentes
    en arábigos.
    '''
    try:
        numeros={'uno':1,'un':1,'una':1,'dos':2,'tres':3,'cuatro':4,'cinco':5,'seis':6,'siete':7,'ocho':8,'nueve':9,'diez':10}
        for key in numeros:
            x = x.lower()
            x=x.replace(key,str(numeros[key]))
        return x
    except:
        return x

def busqueda_por_texto(x):
    '''
    Busca palabras claves asociadas a ambientes, como dormitorio, comedor, living, etc.
    y devuelve el número de veces que estas aparecen.
    '''
    result = 0.0
    valor = 0.0
    try:
        pattern = r"(\d*\s*living\s*cocina\s*comedor|living\s*comedor|living)|(\d*\s*dormitorio)|(\d*\s*comedor)"
        regex = re.compile(pattern, flags = re.IGNORECASE | re.UNICODE)
        grupos = regex.findall(x)
        for grupo in grupos:
            for elemento in grupo:
                regex_dos = re.compile(r'(\d+)', flags = re.IGNORECASE | re.UNICODE)
                cantidad = regex_dos.search(elemento)
                if cantidad is not None:
                    nuevo_valor = float(cantidad.group())
                    if nuevo_valor < 8:
                        valor+=nuevo_valor
            if result == valor:
                result+=1
                valor+=1
            else:
                result=valor
        if result > 0.0:
            return result
        else:
            return np.NaN
    except:
        return np.NaN
def convert_to_int(x):
    return int(x)
def busqueda_moneda(x):
    try:
        pattern = 'U\$D\s*([\d\.]+)'
        regex = re.compile(pattern, flags = re.IGNORECASE | re.UNICODE)
        m = regex.search(x)
        if m:
            return float(m.group(1).replace('.',''))
        else:
            np.NaN
    except:
        return np.NaN

# Parte Ambientes

* Descripción

In [ ]:
df_ambientes.isnull().sum()

* Limpieza de Datos

In [ ]:
df_ambientes_limpio = df_ambientes
df_ambientes_limpio.description=df_ambientes_limpio.description.apply(parsear_numeros)## Reemplaza los números escritos en letras por los arábigos
df_ambientes_limpio.title = df_ambientes_limpio.title.apply(parsear_numeros)
df_ambientes_limpio['rooms_title'] = df_ambientes.title.apply(detectar_ambientes)
df_ambientes_limpio['rooms_description'] = df_ambientes.description.apply(detectar_ambientes)
rooms = df_ambientes.apply(crear_serie_ambientes,1)
## Usar la Serie Rooms



In [ ]:
df_limpio = pd.DataFrame({'Tipo de Propiedad':df.property_type,'Ambientes':rooms,'Titulo':df.title,'Descripcion':df.description})

``` Estado del DataSet al momento ```

In [ ]:
df_limpio.hist()

# Parte Precios

In [ ]:
df_precio.isnull().sum()
## El DataSet nos trae los precios, precios evaluados en ARS y en USD. Usamos los Dolares y nos olvidamos del resto

In [ ]:
set(df_precio.currency)
##Hay valores en Pesos, en Uruguayos, en Peruanos y en Dólares

In [ ]:
df_precio[df_precio.currency =='PEN']
## Los registros que tienen precios en Soles, segun esto pertenece a la Arg

In [ ]:
df_precio[df_precio.currency =='UYU']
## Segun la descripcion, este esta en Mendoza.

In [ ]:
df_precio['precio_en_titulo'] = df_precio.title.apply(busqueda_moneda)
precio = df_precio.apply(crear_serie_precio,1)

In [ ]:
precio.isnull().sum()

In [ ]:
df_precio[df_precio.description.str.contains('USD').isnull()]
## Ya tienen el precio, no me interesa mirarlos por dentro

In [ ]:
df_precio.title.str.contains('dolar').isnull().sum()

- Los datos sin precios los eliminamos

In [ ]:
x=11
df_precio[df_precio.currency.isnull()].iloc[x,-1],df_precio[df_precio.currency.isnull()].iloc[x,-2]

# Parte Superficie

- Por una cuestión de tiempo solo invertimos aquellos casos donde la superficie cubierta era mayor que la total, debería hacerce una busqueda en description o en title en aquellos casos que estén vacíos.

In [ ]:
Superficie = df_precio.apply(lambda x: pd.Series([x.surface_covered_in_m2,x.surface_total_in_m2]) 
                       if x.surface_covered_in_m2 <= x.surface_total_in_m2
                       else pd.Series([x.surface_total_in_m2,x.surface_covered_in_m2]) ,1)

In [ ]:
Superficie.columns = ['surface_covered_in_m2','surface_total_in_m2']

In [ ]:
#Superficie = Superficie.apply(lambda x: pd.Series([x.surface_covered_in_m2,x.surface_covered_in_m2])
#                      if x.surface_covered_in_m2 is None and x.surface_total_in_m2 is not None
#                       else (pd.Series([x.surface_total_in_m2,x.surface_total_in_m2])
#                       if x.surface_covered_in_m2 is not None and x.surface_total_in_m2 is None
#
#else pd.Series([x.surface_covered_in_m2,x.surface_total_in_m2])) ,1)

In [ ]:

Superficie.surface_covered_in_m2.fillna(Superficie.surface_total_in_m2,inplace=True)
Superficie.surface_total_in_m2.fillna(Superficie.surface_covered_in_m2,inplace=True)

In [ ]:
Superficie.columns = ['Superficie Cubierta','Superficie Total']

In [ ]:
## Reagrupamos la Info

In [ ]:
df_limpio['Superficie Cubierta']=Superficie['Superficie Cubierta']
df_limpio['Superficie Total'] = Superficie['Superficie Total']

In [ ]:
df_limpio=df_limpio[['Tipo de Propiedad','Ambientes','Superficie Cubierta','Superficie Total','Titulo','Descripcion']]
df_limpio.sample(5)

# Parte Posición Geográfica

In [ ]:
df_localidad_ex = pd.DataFrame(item for item in df_posicion_geografica.place_with_parent_names.str.split('|'))
df_localidad_ex.isnull().sum()
##Las variables de "zona" 4,5 y 6 estan muy incompletas. la mas recuperable es la 4
#df_posicion_geografica['pais_ex'] = df_localidad_ex[1]
#df_posicion_geografica['provincia_ex'] = df_localidad_ex[2]
#df_posicion_geografica['localidad_ex'] = df_localidad_ex[3]
#df_posicion_geografica['barrio_ex'] = df_localidad_ex[4]
#df_posicion_geografica['zona_ex'] = df_localidad_ex[5]

In [ ]:
set(df_localidad_ex[1]) #Categorias de pais

In [ ]:
set(df_localidad_ex[2]) #Categoria de provincia, BSAS esta dividida en 5 partes

In [ ]:
set(df_localidad_ex[3]) #Localidad

In [ ]:
len(set(df_localidad_ex[4])) #Barrio

In [ ]:
set(df_localidad_ex[5])

In [ ]:
set(df_localidad_ex[6]) #datos vacios

In [ ]:
df_localidad_ex[df_localidad_ex[4] =='coordenadas 34.255511'] 
df_localidad_ex.iloc[7367,4] ='Nordelta'
## El registro 7367 es el unico que tiene este campo y esta en Nordelta segun Properatti

In [ ]:
print('Vacios: ',len(df_localidad_ex[(df_localidad_ex[4] == '') | (df_localidad_ex[4].isnull())][4]))
print('Con datos:',len(df_localidad_ex)-len(df_localidad_ex[(df_localidad_ex[4] == '') | (df_localidad_ex[4].isnull())][4]))

## Hay 80803 vacios, lo mas facil seria tirarlo, lo ideal seria buscar por la lat/lon de donde viene realmente, pero no tenemos
## datos para todos ni el tiempo para hacerlo, en vez de eso lo que hacemos es pegarle a los vacios y los nulos el dato de
## la localidad, que sigue un criterio parecido al que tiene el dataset

In [ ]:
df_posicion_geografica['pais_ex'] = df_localidad_ex[1]
df_posicion_geografica['provincia_ex'] = df_localidad_ex[2]
df_posicion_geografica['localidad_ex'] = df_localidad_ex[3]
df_posicion_geografica['barrio_ex'] = df_localidad_ex[4]

In [ ]:
df_posicion_geografica.place_name.fillna(df_posicion_geografica.localidad_ex,inplace=True)

In [ ]:
df_posicion_geografica[df_posicion_geografica.place_name !=df_posicion_geografica.localidad_ex].head()
##Hay muchas areas en las que no coinciden el place name con la localidad a la que encontramos que pertenece
##En muchos casos coinciden con algun barrio/zona del daaset

In [ ]:
len(df_posicion_geografica[df_posicion_geografica.place_name !=df_posicion_geografica.localidad_ex])
#Hay 35170 datos que no coinciden

In [ ]:
df_posicion_geografica.barrio_ex =df_posicion_geografica.apply(lambda x: x.localidad_ex 
                                       if x.barrio_ex == '' or x.barrio_ex ==None 
                                       else x.barrio_ex,1)

In [ ]:
df_posicion_geografica[df_posicion_geografica.barrio_ex == ''].title.iloc[0]

In [ ]:
df_posicion_geografica.head()

In [ ]:
df_limpio['Pais'] = df_posicion_geografica['pais_ex']
df_limpio['Provincia'] = df_posicion_geografica['provincia_ex']
df_limpio['Localidad'] = df_posicion_geografica['localidad_ex']
df_limpio['Barrio'] = df_posicion_geografica['barrio_ex']

In [ ]:
df_limpio=df_limpio[['Tipo de Propiedad','Ambientes','Superficie Cubierta','Superficie Total','Pais','Provincia','Localidad','Barrio','Titulo','Descripcion']]

In [ ]:
df_limpio.head()

In [ ]:
df_precio.info()

In [ ]:
df_limpio['Precio Metro Cuadrado'] = df_precio.price_usd_per_m2               
df_limpio['Precio'] = df_precio.price_aprox_usd

In [ ]:
df_limpio=df_limpio[['Precio Metro Cuadrado','Precio','Tipo de Propiedad','Ambientes','Superficie Cubierta','Superficie Total','Pais','Provincia','Localidad','Barrio','Titulo','Descripcion']]

In [ ]:
df_limpio.head()

# Modelado

In [ ]:
df_limpio.iloc[205]

In [ ]:
def recalcular_precio_metro_cuadrado(x):
    try:
        if np.isnan(np.array(x['Precio Metro Cuadrado'])):
            if not(np.isnan(np.array(x['Superficie Total'])) or np.isnan(np.array(x['Precio']))):
                return x['Precio']/x['Superficie Total']
    except:
        return np.nan
df_limpio['Precio Metro Cuadrado'].fillna(df_limpio.apply(recalcular_precio_metro_cuadrado,1),inplace=True)

In [ ]:
df_limpio.isnull().sum()

In [ ]:
df_definitivo=df_limpio.drop(columns=['Descripcion','Titulo']).dropna()


In [ ]:
df_definitivo.

In [ ]:
df_definitivo.corr()

In [ ]:
df_dummies =pd.get_dummies(df_definitivo,columns=['Localidad','Tipo de Propiedad'], drop_first=True)

In [ ]:
df_dummies.corr()

In [ ]:
model =LinearRegression(fit_intercept = True)
modelRidge=Ridge()
modelLasso=Lasso()
y = df_dummies['Precio Metro Cuadrado']
X = df_dummies.drop(columns=['Precio Metro Cuadrado','Pais','Barrio','Provincia'])
Xtrain,Xtest,ytrain,ytest= train_test_split(X,y,\
                                     random_state=1)

In [ ]:
Xtrain.shape,ytrain.shape

In [ ]:
Xtest.shape,ytest.shape

In [ ]:
model.fit(Xtrain,ytrain)
modelRidge.fit(Xtrain,ytrain)
modelLasso.fit(Xtrain,ytrain)

In [ ]:
model.intercept_

In [ ]:
df_coef=pd.DataFrame(model.coef_).T
df_coef.columns = df_dummies.drop(columns=['Precio Metro Cuadrado','Pais','Barrio','Provincia']).columns

In [ ]:
df_coef

In [ ]:
ypred=model.predict(Xtest)
ypredRidge=modelRidge.predict(Xtest)
ypredLasso=modelLasso.predict(Xtest)

In [ ]:
print('Lineal: ',metrics.r2_score(ytest,ypred))
print('Ridge: ',metrics.r2_score(ytest,ypredRidge))
print('Lasso: ',metrics.r2_score(ytest,ypredLasso))

